# Quantum Synth: the implemented quantum circuits #

The tested quantum circuits to be provided to the synthesizer are reported in this notebook. From a programming point of view, they are methods adding quantum gates to a quantum circuit provided as input file. In the tested application cases, each circuit has been applied to the quantum register when it is equal to |0000>, then the quantum circuits are provided to the ChooseBackEnd method for executing quantum circuits. **Pay attention: quantum circuits are usually not executed here, with the exception of a particular case which will be described into detail.**

In [2]:
from qiskit import *
import numpy as np
import random
import sys
from backends_select import ChooseBackEnd

## Quantum Circuit for generating one basis state starting from |0000> ##

This quantum circuit generates one basis state between |0000> and |1011> by simply applying X gates to the qubits that must be equal to 1.

In [3]:
def GenerateCircuitSingleNote(circuit, note_id):
    if (note_id >= 12):
        sys.exit("Note must be an integer smaller than 11 and larger (or equal) to 0.")
    bitstring = str(bin(note_id)[2:])
    bitstring = "0"*(4-len(bitstring))+bitstring
    for i in range(len(bitstring)):
        if bitstring[len(bitstring)-1-i] == "1":
            circuit.x(i)

In [4]:
qc = QuantumCircuit(4,4)
GenerateCircuitSingleNote(qc, 7)
ChooseBackEnd(qc, 'statevector_simulator')

['0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '1.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000']

## Bell state generation between two qubits ##

The two qubits to be entangled are provided as input parameters; moreover, it is possible to obtain a $\vert \Phi>$ or a $\vert \Psi>$ entangled state. Pay attention: in the current version of the Quantum synthesizer the phase is not employed, so we cannot distinguish between states as $\vert \Phi^{+}>$ and $\vert \Phi^{-}>$. 

In [6]:
def BellStateGenerationTwoQubits(quantumCircuit, firstQubit=0, secondQubit=1, specificEntangledState="Phi"):
    if specificEntangledState == "Phi":
        quantumCircuit.h(firstQubit)
        quantumCircuit.cx(firstQubit, secondQubit)
    elif specificEntangledState == "Psi":
        quantumCircuit.h(firstQubit)
        quantumCircuit.x(secondQubit)
        quantumCircuit.cx(firstQubit, secondQubit)

In [7]:
qc = QuantumCircuit(4,4)
BellStateGenerationTwoQubits(qc, 0, 1)
ChooseBackEnd(qc, 'qasm_simulator')

range(0, 4)


['0.500',
 '0.000',
 '0.000',
 '0.500',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000']

## Equal superposition of basis states ##

Basis states to be in superposition are provided inside a list, which is the second parameter of the method. In this particular case, the initialize method of the quantum circuit class is employed: it inserts the quantum circuit required for obtaining the desired state starting from $\vert 00 \cdots 0>$. It is also possible to create a list of random states to be in equal superposition with the function ChooseEqualSuperpositionRandom.

In [14]:
def ChooseEqualSuperposition(quantumCircuit, states):
    desiredVector = np.zeros(2**quantumCircuit.n_qubits)
    flag = 1
    for k in states:
        if 0 <= k <= 11:
            desiredVector[k] = 1/np.sqrt(len(states))
            flag = flag*1
        else:
            flag = flag*0
    if flag == 1:
        quantumCircuit.initialize(desiredVector, range(4))
        
def ChooseEqualSuperpositionRandom(quantumCircuit):
    randomNumberOfNotes = np.random.randint(2,13)
    listModes = list(range(12))
    listToSuperimpose = []
    for i in range(randomNumberOfNotes):
        tmp = random.choice(listModes)
        listToSuperimpose.append(tmp)
        listModes.remove(tmp)
    ChooseEqualSuperposition(quantumCircuit, listToSuperimpose)

In [35]:
qc = QuantumCircuit(4,4)
ChooseEqualSuperposition(qc, [1, 4, 7])
ChooseBackEnd(qc, 'qasm_simulator')

range(0, 4)


['0.000',
 '0.337',
 '0.000',
 '0.000',
 '0.325',
 '0.000',
 '0.000',
 '0.338',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000']

## Hadamard gates ##

Hadamard gates are applied to qubits which are in a list to be provided to the function Hadamard.

In [16]:
def Hadamard(quantumCircuit, listOfQubits):
    for k in listOfQubits:
        if 0 <= k <= quantumCircuit.n_qubits:
            quantumCircuit.h(k)

In [20]:
qc = QuantumCircuit(4,4)
Hadamard(qc, [0, 2])
ChooseBackEnd(qc, 'statevector_simulator')

['0.250',
 '0.250',
 '0.000',
 '0.000',
 '0.250',
 '0.250',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000',
 '0.000']

## Random rotation to each qubit ##

The $U3(\theta, \phi, \lambda)$ gate is applied to each qubit. For each gate, the three paramters are chosen randomly.

In [21]:
def RandomRotation(quantumCircuit):
    for k in range(quantumCircuit.n_qubits):
        quantumCircuit.u3(q=k, theta = np.random.random()*2*np.pi, phi = np.random.random()*np.pi, lam = np.random.random()*np.pi)

In [23]:
qc = QuantumCircuit(4,4)
RandomRotation(qc)
ChooseBackEnd(qc, 'qasm_simulator')

range(0, 4)


['0.026',
 '0.009',
 '0.080',
 '0.029',
 '0.014',
 '0.003',
 '0.036',
 '0.011',
 '0.105',
 '0.036',
 '0.303',
 '0.107',
 '0.045',
 '0.015',
 '0.132',
 '0.049']

## Grover's search: single measurement case ##

In the following, the methods for designing the four-qubit Grover's search algorithm are reported. It is possible to set the value to be amplified by providing to the Grover method the target as binary string. The oracle has been designed and it required the use of two ancilla qubits, since the Controlled-Controlled-Controlled-$Z$ gate is not natively available in Qiskit. In order to avoid problems with the measurement results, it has been required to provide as input parameter of the functions the initialLength, *i.e.* the length of the quantum register without the ancillas.

In [25]:
def __multiplecz(quantumCircuit, target, initialLength):
    quantumCircuit.ccx(0,1, initialLength)
    for k in range(2, initialLength-1):
        quantumCircuit.ccx(k, initialLength+k-2, initialLength+k-1)
    quantumCircuit.cz(quantumCircuit.n_qubits-1, initialLength-1)
    for k in reversed(range(2, initialLength-1)):
        quantumCircuit.ccx(k, initialLength+k-2, initialLength+k-1)
    quantumCircuit.ccx(0,1, initialLength)
    
def Grover(quantumCircuit, target, initialLength):
    for k in range(initialLength):
        quantumCircuit.h(k)
    ancillaQubit = QuantumRegister(2)
    quantumCircuit.add_register(ancillaQubit)
    for n in range(int(np.round(np.pi/4*np.sqrt(2**initialLength)))):
        
        for singleBit in range(initialLength):
            if target[initialLength-singleBit-1] == '0':
                quantumCircuit.x(singleBit)
        __multiplecz(quantumCircuit, target, initialLength)
        for singleBit in range(initialLength):
            if target[initialLength-singleBit-1] == '0':
                quantumCircuit.x(singleBit)
                
        for qubit in range(initialLength):
            quantumCircuit.h(qubit)
            quantumCircuit.x(qubit)
        __multiplecz(quantumCircuit, target, initialLength)
        for qubit in range(initialLength):
            quantumCircuit.x(qubit)
            quantumCircuit.h(qubit)

In [27]:
qc = QuantumCircuit(4,4)
targ = format(10, '#06b')[2:]
Grover(qc,targ,4)
ChooseBackEnd(quantumCircuit=qc, backendType="statevector_simulator", qubitsToBeMeasured=range(4))

['0.003',
 '0.003',
 '0.003',
 '0.003',
 '0.003',
 '0.003',
 '0.003',
 '0.003',
 '0.003',
 '0.003',
 '0.961',
 '0.003',
 '0.003',
 '0.003',
 '0.003',
 '0.003']

## Grover's search: multiple measurement ##

A possible "music interpretation" of the Grover's search algorithm is an interative volume amplification of one or more notes starting from noise generated by all notes played with low volume at the same time. In order to do that, we wrote a function that permits to get the probabilities at each step of the algorithm, *i.e.* after each amplitude amplification (phase oracle + inversion about the mean).
This method is slightly different from the previous ones, since it does not only create the quantum circuit, but it also launches the simulation of each intermediate step. This is due to the fact that the QASM simulator requires to measure the quantum register in order to get the probabilities, differently from the statevector_simulator having the method get_statevector, thus requiring to build a new circuit to be measured at each step of the amplitude amplification procedure.

In [28]:
def AmplitudeAmplification(quantumCircuit, target, initialLength, numIterations):
    for k in range(initialLength):
        quantumCircuit.h(k)
    ancillaQubit = QuantumRegister(2)
    quantumCircuit.add_register(ancillaQubit)
    for n in range(numIterations):
        for singleBit in range(initialLength):
            if target[initialLength - singleBit - 1] == '0':
                quantumCircuit.x(singleBit)
        __multiplecz(quantumCircuit, target, initialLength)
        for singleBit in range(initialLength):
            if target[initialLength - singleBit - 1] == '0':
                quantumCircuit.x(singleBit)

        for qubit in range(initialLength):
            quantumCircuit.h(qubit)
            quantumCircuit.x(qubit)
        __multiplecz(quantumCircuit, target, initialLength)
        for qubit in range(initialLength):
            quantumCircuit.x(qubit)
            quantumCircuit.h(qubit)


def GroverSequence(target, initialLength,backendType,RealDeviceName,noisePresent):
    iterations = []
    for k in range(4):
        temporaryQuantumCircuit = QuantumCircuit(initialLength, initialLength)
        AmplitudeAmplification(temporaryQuantumCircuit, target, initialLength, k)
        #             listForMusic = ChooseBackEnd(music, backendType=mystr[0], qubitsToBeMeasured=range(4),
        #             numberShots=int(mystr[3]), noisePresent=True, RealDeviceName=mystr[1])

        iterations.append(ChooseBackEnd(quantumCircuit=temporaryQuantumCircuit, noisePresent=noisePresent,backendType=backendType,qubitsToBeMeasured=range(4),RealDeviceName=RealDeviceName))
        # ChooseBackEnd(quantumCircuit=temporaryQuantumCircuit, noisePresent=True,backendType=backendType,qubitsToBeMeasured=range(4),RealDeviceName=RealDeviceName)
        del (temporaryQuantumCircuit)

    return iterations

In [33]:
targ = format(10, '#06b')[2:]
GroverSequence(targ,4,'statevector_simulator','ibmq_ourense',False)

[['0.063',
  '0.063',
  '0.063',
  '0.062',
  '0.063',
  '0.062',
  '0.062',
  '0.062',
  '0.063',
  '0.062',
  '0.062',
  '0.062',
  '0.062',
  '0.062',
  '0.062',
  '0.062'],
 ['0.035',
  '0.035',
  '0.035',
  '0.035',
  '0.035',
  '0.035',
  '0.035',
  '0.035',
  '0.035',
  '0.035',
  '0.473',
  '0.035',
  '0.035',
  '0.035',
  '0.035',
  '0.035'],
 ['0.006',
  '0.006',
  '0.006',
  '0.006',
  '0.006',
  '0.006',
  '0.006',
  '0.006',
  '0.006',
  '0.006',
  '0.908',
  '0.006',
  '0.006',
  '0.006',
  '0.006',
  '0.006'],
 ['0.003',
  '0.003',
  '0.003',
  '0.003',
  '0.003',
  '0.003',
  '0.003',
  '0.003',
  '0.003',
  '0.003',
  '0.961',
  '0.003',
  '0.003',
  '0.003',
  '0.003',
  '0.003']]